# Demo de "First Order Motion Model for Image Animation"  para animar Caras (se muestran todas las de "sample") con el mismo video
fuente: https://github.com/AliaksandrSiarohin/first-order-model


1) Preparar el entorno:

In [ ]:
#@title Clonar paquete first-order-model
!git clone https://github.com/AliaksandrSiarohin/first-order-model
print("\nFirst Order Motion Model  instalado:")
%cd first-order-model
!ls .

fatal: destination path 'first-order-model' already exists and is not an empty directory.

First Order Motion Model  instalado:
/content/first-order-model
animate.py	 demo.ipynb	    logger.py	       requirements.txt
augmentation.py  demo.py	    modules	       run.py
config		 Dockerfile	    __pycache__        sup-mat
crop-video.py	 frames_dataset.py  README.md	       sync_batchnorm
data		 LICENSE.md	    reconstruction.py  train.py


In [ ]:
#@title Cargar Librerías
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
from demo import make_animation
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")

print("Librerías cargadas.")

Librerías cargadas.


In [ ]:
#@title Montar el drive
from google.colab import drive
drive.mount('/content/gdrive')

dir = '/content/gdrive/MyDrive/IA/-pruebaDeepFake/'
dirDatos = dir + 'demoCaras/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#@title Cargar checkpoint del modelo ya entrenado
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path= './config/vox-256.yaml', 
                            checkpoint_path= dir + '/first-order-motion-model/vox-cpk.pth.tar')

print("Modelo cargado.")

Modelo cargado.


In [ ]:
#@title Definir funciones auxiliares
# función para mostrar imagen con video
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

def procesa(imagenFuente, videoFuente):
    source_image = imageio.imread(imagenFuente)
    driving_video = imageio.mimread(videoFuente, memtest=False)

    #Resize image and video to 256x256
    source_image = resize(source_image, (256, 256))[..., :3]
    driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

    predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                                adapt_movement_scale=True)

    return HTML(display(source_image, driving_video, predictions).to_html5_video())

print("Funciones auxiliares definidas.")    

Funciones auxiliares definidas.


2) Procesar las imágenes:


In [ ]:
import os

video_seleccionado =  "sample2-dicaprio.mp4" #@param [ 'sample1-obama.mp4', 'sample2-dicaprio.mp4', 'jackNicholson.mp4' ] {allow-input: true}
videoFuente = dirDatos + video_seleccionado

arDir = os.listdir(dirDatos)
imagenes = []
for ar in arDir:
  if ar.find('.png')>=0 or ar.find('.jpg')>=0 or ar.find('.jpeg')>=0:
    if ar.find('sample')>=0:
      imagenes.append( ar )

print(videoFuente)
print(imagenes)

/content/gdrive/MyDrive/IA/-pruebaDeepFake/demoCaras/sample2-dicaprio.mp4
['sample1-shrek.png', 'sample2-chucky.png', 'sample3-it.png', 'sample4-socrates.png', 'sample5-putin.png']


In [ ]:
procesa(dirDatos + imagenes[0], videoFuente)

100%|██████████| 444/444 [00:16<00:00, 26.24it/s]


In [ ]:
procesa(dirDatos + imagenes[1], videoFuente)

100%|██████████| 444/444 [00:16<00:00, 26.19it/s]


In [ ]:
procesa(dirDatos + imagenes[2], videoFuente)

100%|██████████| 444/444 [00:17<00:00, 25.63it/s]


In [ ]:
procesa(dirDatos + imagenes[3], videoFuente)

100%|██████████| 444/444 [00:17<00:00, 25.38it/s]


In [ ]:
procesa(dirDatos + imagenes[4], videoFuente)

100%|██████████| 444/444 [00:17<00:00, 25.18it/s]
